In [8]:
import os
from shapely import wkt
import pandas as pd
import geopandas as gpd
import json

In [17]:
json_path = "/Users/Administrator/Documents/University/Year 3/2b/Thesis/Bachelor-Thesis/data/raw/Elevation/kaartbladindex.json"

nitrate_dir = "/Users/Administrator/Documents/University/Year 3/2b/Thesis/Bachelor-Thesis/data/clean/well_chem_data/for_Alignment/utrecht_well_chem_combined_1.csv"
nitrate_df = pd.read_csv(nitrate_dir, parse_dates=['Date'])

In [18]:
nitrate_df['geometry'] = nitrate_df['geometry'].apply(wkt.loads)
nitrate_gdf = gpd.GeoDataFrame(nitrate_df, geometry='geometry', crs="EPSG:4326")

In [19]:
# 2. Reproject nitrate points to AHN tile CRS
nitrate_gdf = nitrate_gdf.to_crs("EPSG:28992")

# 3. Load kaartbladindex (tile polygons)
with open(json_path) as f:
    data = json.load(f)

tiles_gdf = gpd.GeoDataFrame.from_features(data["features"], crs="EPSG:28992")

# 4. Spatial join — keep only AHN tiles that intersect nitrate wells
intersecting_tiles = gpd.sjoin(tiles_gdf, nitrate_gdf, how="inner", predicate="intersects")

# 5. Drop duplicates (same tile may intersect multiple wells)
intersecting_tiles = intersecting_tiles.drop_duplicates(subset=["name"])

# 6. Get URLs for download (you can modify to switch DSM to DTM if needed)
print(intersecting_tiles[["name", "url"]])


            name                                                url
699  M_31HN2.tif  https://service.pdok.nl/rws/ahn/atom/downloads...
702  M_32AZ2.tif  https://service.pdok.nl/rws/ahn/atom/downloads...
703  M_32CN2.tif  https://service.pdok.nl/rws/ahn/atom/downloads...
708  M_32AN2.tif  https://service.pdok.nl/rws/ahn/atom/downloads...
710  M_32BZ1.tif  https://service.pdok.nl/rws/ahn/atom/downloads...
711  M_32CZ1.tif  https://service.pdok.nl/rws/ahn/atom/downloads...
712  M_32DZ2.tif  https://service.pdok.nl/rws/ahn/atom/downloads...
713  M_32DN1.tif  https://service.pdok.nl/rws/ahn/atom/downloads...
714  M_32DN2.tif  https://service.pdok.nl/rws/ahn/atom/downloads...
716  M_32CZ2.tif  https://service.pdok.nl/rws/ahn/atom/downloads...
717  M_32CN1.tif  https://service.pdok.nl/rws/ahn/atom/downloads...
718  M_32DZ1.tif  https://service.pdok.nl/rws/ahn/atom/downloads...
724  M_32GZ2.tif  https://service.pdok.nl/rws/ahn/atom/downloads...
726  M_32GN1.tif  https://service.pdok.nl/rws/ah